In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import random as rd

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Crear datasets

In [66]:
df = pd.DataFrame({
    'id_libro': range(1,101),
    'nombre':["libro"+str(i) for i in range(1,101)], 
    'género': rd.choices(['terror', 'suspenso', 'ciencia ficcion', 'anime','romantico','maravilloso','otros'],k=100),
    'autor':rd.choices(["autor"+str(i) for i in range(1,30)],k=100)
})
libros = sqlContext.createDataFrame(df).rdd.map(tuple).cache()

In [67]:
df = pd.DataFrame({
 'id_venta': range(1,1001),
 'id_libro': rd.choices(range(1,101),k=1000),
 'dia_venta': pd.date_range(start='2020-07-21',freq='H',periods=1000).strftime("%d"),
 'mes_venta': pd.date_range(start='2020-07-21',freq='H',periods=1000).strftime("%B"),
 'año_venta': pd.date_range(start='2020-07-21',freq='H',periods=1000).strftime("%Y"),
 'hora_venta':pd.date_range(start='2020-07-21',freq='H',periods=1000).strftime("%H"),
 'precio': rd.choices(range(100,1000),k=1000)
})
ventas = sqlContext.createDataFrame(df).rdd.map(tuple).cache()

In [68]:
libros.take(10)

[(1, 'libro1', 'maravilloso', 'autor11'),
 (2, 'libro2', 'maravilloso', 'autor15'),
 (3, 'libro3', 'ciencia ficcion', 'autor9'),
 (4, 'libro4', 'suspenso', 'autor24'),
 (5, 'libro5', 'maravilloso', 'autor26'),
 (6, 'libro6', 'anime', 'autor11'),
 (7, 'libro7', 'maravilloso', 'autor29'),
 (8, 'libro8', 'suspenso', 'autor11'),
 (9, 'libro9', 'maravilloso', 'autor8'),
 (10, 'libro10', 'terror', 'autor26')]

In [69]:
ventas.take(3)

[(1, 24, '21', 'July', '2020', '00', 936),
 (2, 13, '21', 'July', '2020', '01', 518),
 (3, 54, '21', 'July', '2020', '02', 428)]

a) Indicar el género con más ventas de agosto de 2020. 

In [70]:
ventas_agosto = ventas.filter(lambda x: (x[3] == 'August' and x[4]=='2020')).map(lambda x: (x[1],1))
ventas_agosto.take(3)

[(62, 1), (95, 1), (3, 1)]

In [71]:
libros_cat = libros.map(lambda x: (x[0],x[2]))
libros_cat.take(3)

[(1, 'maravilloso'), (2, 'maravilloso'), (3, 'ciencia ficcion')]

In [81]:
generos_vendidos = libros_cat.join(ventas_agosto).map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y).cache()
generos_vendidos.reduce(lambda x,y: x if x[1]>y[1] else y)

('maravilloso', 177)

In [102]:
five_top_gen = dict(generos_vendidos.takeOrdered(5,lambda x: -x[1]))
books_of_five_top_gen = libros.filter(lambda x: x[2] in five_top_gen).map(lambda x: (x[0],x[1]))
books_of_five_top_gen.take(3)

[(1, 'libro1'), (2, 'libro2'), (3, 'libro3')]

In [136]:
ventas_periodo = ventas.filter(lambda x: (x[4]=='2020' and (x[3] == 'August' or x[3]=='July'))).map(lambda x: (x[1],x[3]))
books_of_five_top_gen.join(ventas_periodo)\
    .map(lambda x: ((x[1][0],x[1][1]),1))\
    .reduceByKey(lambda x,y: x+y)\
    .map(lambda x: (x[0][0],(x[1],0) if x[0][1] == 'July' else (0,x[1]) ))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .reduce(lambda x,y: x if x[1][1]-x[1][0] > y[1][1]-y[1][0] else y)[0]

'libro74'